In [1]:
import sys
sys.path.append('C:\\Users\\szb37\\My Drive\\Projects\\ADAPT\\ADAPT codebase\\')
import src.config as config
import src.folders as folders
import src.power_calc as power
import pandas as pd
import os

prefix='adapt'
n_trials = 200
sample_size = 400

Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


In [2]:
### Generte mock trial data

# Note on the SDs: in the POP data the SD of the dose guess is:
# - 3.6mg in the Control arm
# - 11.6mg in the Treatment arm
# For the simulation I use (3.6+11.6)/2~7.6mg
df_trials=[]

scenario = 'diff0mg'
df_trial = power.DataGeneration.get_df_trials(
    scenario = scenario,
    n_trials = n_trials, 
    sample_size = sample_size, 
    mean_C = 10.5, 
    mean_T = 10.5,        
    sd = 11.1, 
    confs = config.confs,
    )
df_trials.append(df_trial)

scenario = 'diff2mg'
df_trial = power.DataGeneration.get_df_trials(
    scenario = scenario,
    n_trials = n_trials, 
    sample_size = sample_size, 
    mean_C = 9.5, 
    mean_T = 11.5,        
    sd = 11.1, 
    confs = config.confs,
    )
df_trials.append(df_trial)

scenario = 'diff4mg'
df_trial = power.DataGeneration.get_df_trials(
    scenario = scenario,
    n_trials = n_trials, 
    sample_size = sample_size, 
    mean_C = 8.5, 
    mean_T = 12.5,        
    sd = 11.1, 
    confs = config.confs,
    )
df_trials.append(df_trial)

scenario = 'diff6mg'
df_trial = power.DataGeneration.get_df_trials(
    scenario = scenario,
    n_trials = n_trials, 
    sample_size = sample_size, 
    mean_C = 7.5, 
    mean_T = 13.5,        
    sd = 11.1, 
    confs = config.confs,
    )
df_trials.append(df_trial)

scenario = 'diff8mg'
df_trial = power.DataGeneration.get_df_trials(
    scenario = scenario,
    n_trials = n_trials, 
    sample_size = sample_size, 
    mean_C = 6.5, 
    mean_T = 14.5,        
    sd = 11.1, 
    confs = config.confs,
    )
df_trials.append(df_trial)

### Concatanate and save 
df_trials = pd.concat(df_trials, ignore_index=True)
df_trials.to_csv(os.path.join(folders.power, f'{prefix}_trials.csv'), index=False)

Gen diff8mg trial data: 100%|██████████| 200/200 [00:00<00:00, 250.03it/s]


In [3]:
### Calculate CIs according to various methods - can take few mins to run
df_cis = power.Stats.get_df_cis(
    df_trials = pd.read_csv(os.path.join(folders.power, f'{prefix}_trials.csv')),
    sample_sizes = [60, 80, 100, 120, 140, 160, 180, 200, 240, 280, 320, 360, 400],
    #sample_sizes = [60, 100, 140, 180, 220, 260, 300, 400],
    rope_cgr=0.14, 
    rope_bbi=0.2, 
    rope_gmg=5,
    rope_gmgc=5,)        
df_cis.to_csv(os.path.join(folders.power, f'{prefix}_cis.csv'), index=False)

Calc CIs: 2600it [00:52, 49.14it/s]
Calc CIs: 2600it [00:52, 49.74it/s]
Calc CIs: 2600it [00:52, 49.41it/s]
Calc CIs: 2600it [00:51, 50.74it/s]
Calc CIs: 2600it [00:50, 51.34it/s]


In [4]:
### Calc averages across scenarios / sample sizes 
df_power = power.Power.get_df_power(
    df_cis = pd.read_csv(os.path.join(folders.power, f'{prefix}_cis.csv')),
    reduced_output=False)
df_power.to_csv(os.path.join(folders.power, f'{prefix}_power.csv'), index=False)